In [192]:
import json
import os
from random import randint

def randomize_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jams"):
            file_path = os.path.join(folder_path, filename)
            randomBPM(file_path)

def randomBPM(file_path):
    divmulgate = randint(0, 1)
    if divmulgate == 0:
        bpm = 1 + (randint(0, 25) * 0.01)
    else:
        bpm = 1 - (randint(0, 25) * 0.01)

    with open(file_path, 'r+') as f:
        jam = json.load(f)
        
        if 'file_metadata' in jam and 'duration' in jam['file_metadata']:
                jam['file_metadata']['duration'] *= bpm

        for anno in jam['annotations']:
            if anno['namespace'] == 'note_midi':
                for note in anno['data']:
                    note['duration'] *= bpm
                    note['time'] *= bpm

            elif anno['namespace'] in ['pitch_contour', 'beat_position', 'tempo']:
                if 'time' in anno['data']:  # Check if 'time' exists
                    for i in range(len(anno['data']['time'])):
                        anno['data']['time'][i] *= bpm
                elif 'data' in anno:  # Check if 'data' exists
                    for i in range(len(anno['data'])):
                        anno['data'][i]['time'] *= bpm
                        if anno['namespace'] == 'tempo':  # Additional processing for 'tempo' namespace
                            anno['data'][i]['value'] *= bpm

        # Move the file pointer to the beginning of the file
        f.seek(0)
        # Write the modified JSON back to the file
        json.dump(jam, f, indent=4)
        # Truncate the remaining content (in case the new content is shorter than the old content)
        f.truncate()

# Usage example:
folder_path = 'jams/'
randomize_files_in_folder(folder_path)
